<div class="alert alert-block alert-info"> <b>NOTE</b> 

<div>
    This is a duplicate of notebook <b>04_prediction.ipynb </b>
    in this notebook we load the normalization weights to realize prediction with the new models.
</div>
</div>

In [1]:
import os
import re
from PIL import Image
from auxiliary_functions import load_joblib

# For testing data

In [2]:
# Load the images 
model = 'lgbm'
root_dir = f'/Users/luisescobar/Documents/Thesis/Images/TOP5_exp/{model}'
save_dir = f'/Users/luisescobar/Documents/Thesis/Combined_images/TOP5_exp/{model}'
path_to_read2 = '/Users/luisescobar/Documents/Thesis/DataSets/Dictionary/03_01_Train_Val_Test'

filename_clot = 'blocking_20.pkl'
filename_noclot = 'no_blocking_20.pkl'

clot_dict = load_joblib(path_to_read2, filename_clot)

# For one sample

In [3]:
def concat_images(image_paths, name, save_dir, direction='horizontal'):
    # Open images and store them in a list
    images = [Image.open(path) for path in image_paths]
    
    # Calculate total width and height based on the chosen direction
    if direction == 'horizontal':
        total_width = sum(img.width for img in images)
        max_height = max(img.height for img in images)
        combined_image = Image.new('RGB', (total_width, max_height))
        
        # Paste images side-by-side
        x_offset = 0
        for img in images:
            combined_image.paste(img, (x_offset, 0))
            x_offset += img.width
            
    elif direction == 'vertical':
        max_width = max(img.width for img in images)
        total_height = sum(img.height for img in images)
        combined_image = Image.new('RGB', (max_width, total_height))
        
        # Paste images one below another
        y_offset = 0
        for img in images:
            combined_image.paste(img, (0, y_offset))
            y_offset += img.height
    
    combined_image.save(f'{save_dir}/combine_images_{name}.png', format='PNG') 
    return combined_image



In [4]:
clot_dict.keys()

dict_keys(['df_1', 'df_923', 'df_680-df_681-concat', 'df_654', 'df_311', 'df_344-df_345-df_346-concat', 'df_902', 'df_486', 'df_513', 'df_62', 'df_653', 'df_783', 'df_790', 'df_247', 'df_190', 'df_751', 'df_764', 'df_859', 'df_115'])

We concatenate the individual images from 04_00_prediction-best-practices.ipynb.

Now a single image will contain the response of the predictor on the same treatments for the lags we are using

In [5]:
for name in clot_dict.keys():
    image_paths = []
    
    pattern = re.compile(rf"^{re.escape(name)}(_|\.).*")
    
    for dirpath, _, filenames in os.walk(root_dir):
        for filename in filenames:
            if pattern.match(filename):
            #if name in filename:  # Check for substring 'xgboost_model' in filename
                image_paths.append(os.path.join(dirpath, filename))

    image_paths.sort()
    combined_image = concat_images(image_paths, name, save_dir)

We create a single image containing all the images from the previous step. We want to compare the behavior of the predictor for the different treatments and lag values

In [6]:
final_paths = []
for dirpath, _, filenames in os.walk(save_dir):
    for filename in filenames:
        if ('.png' in filename) and (filename != 'combine_images_final.png'):
            final_paths.append(f'{save_dir}/{filename}')

In [7]:
combined_image = concat_images(final_paths, 'final', save_dir, direction='vertical')